# Compute opacities

In [ ]:
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import optool

if '..' not in sys.path:
    sys.path.append('..')

from helper import Capturing, apply_araa_style, output_dir, read_paletton_text
import helper

# Read the paletton color file
colors = helper.set_paletton_colors([1,2,3,0])

In [ ]:
settings = {}
settings['DSHARP'] = 'astrosil 0.3291 c-org 0.3966 fes 0.0743 h2o-w 0.2000 -mie'
settings['DSHARP-ZBED'] = 'astrosil 0.3291 c 0.3966 fes 0.0743 h2o-w 0.2000 -dhs'
settings['DIANA'] = '-dhs'

Note that there were issues with optool that magically disappeared. here's the test in case it happens again. The first entry of fp12 should be the same and the rest not different by many orders of magnitude.
```python
p1 = optool.particle(f'optool -mie -dsharp -a 1.0 1000.0 3.5 100 -l 1250 -s 180')
p2 = optool.particle(f'optool -mie -dsharp -a 1.0 1000.0 3.5 100 -l 1250 3000 5 -s 180')

for p in [p1, p2]:
    print(f'lam = {p.lam}')
    print(f'fp12 = {p.f12[0, :, 89]}')
```

In [ ]:
na = 200
a = np.geomspace(1e-3, 1e2, na)
amin = 1.e-5
lam = [0.125, 0.3]
nlam = len(lam)

result = {}

for q in [2.5, 3.5]:
    
    res = {}
    result[str(q)] = res
    out = Capturing()
    
    for key in settings.keys():
    
        d = {}
        res[key] = d
    
        d['lam']   = lam
        d['amax']  = a
        d['g_sca'] = np.zeros([na, nlam])
        d['k_abs'] = np.zeros([na, nlam])
        d['k_sca'] = np.zeros([na, nlam])
        d['omega'] = np.zeros([na, nlam])
        d['omeff'] = np.zeros([na, nlam])
        d['Pscat'] = np.zeros([na, nlam])
        
        for i, amax in tqdm(enumerate(a), total=na):
            p = optool.particle(f'optool {settings[key]} -a {amin * 1e4} {amax * 1e4} {q} {10 + 5 * i} -l {1e4 * lam[0]} {1e4 * lam[-1]} {nlam} -s',
                                silent=True)
            
            # supress the warning in the conversion
            with out:
                p.scatnorm('r')
    
            i895 = np.abs(p.scatang-89.5).argmin()
            if not np.isclose(p.scatang[i895], 89.5) or (not np.isclose(p.scatang[i895+1], 90.5)):
                warnings.warn('scattering angles not exactly at 89.5º and 90.5º' )
                
            d['k_abs'][i, :] = p.kabs
            d['k_sca'][i, :] = p.ksca
            d['g_sca'][i, :] = p.gsca
            d['omega'][i, :] = p.ksca / p.kext
            d['omeff'][i, :] = p.ksca * (1 - p.gsca) / (p.kabs + p.ksca * (1 - p.gsca))
            d['Pscat'][i, :] = np.abs(p.f12[0, :, i895:i895+2] / p.f11[0, :, i895:i895+2]).mean(-1)

# Plot

In [ ]:
ilam = 0

f, axs = plt.subplots(3, 2, sharex=True, sharey='row', figsize=(8, 9), gridspec_kw={'hspace':0.05, 'wspace':0.05})

for iq, q in enumerate(result.keys()):

    res = result[q]    
    
    
    ##### plot the kappas (mass absorption/scattering coefficients) #######
    ax = axs[0, iq]
    
    for i,key in enumerate(settings.keys()):
        line1, = ax.loglog(a, res[key]['k_abs'][:,ilam], label=key)
        line2, = ax.loglog(a, res[key]['k_sca'][:,ilam], '--', c=line1.get_color(), alpha=0.5)
    
    ax.plot([],[], 'k', ls=line1.get_linestyle(), label=r'$\bar\kappa_{abs}$')
    ax.plot([],[], 'k', ls=line2.get_linestyle(), label=r'$\bar\kappa_{sca}$', alpha=line2.get_alpha())
    ax.legend(fontsize='small', frameon=False, loc='lower center')
    
    ax.set_ylim(2e-2, 60)
    ax.set_ylabel(r'$\bar \kappa$ [cm$^2$/g]')
    
    ax.set_title(f'$q = {q}$')
    
    
    ####### plot the spectral index beta #######
    ax = axs[1, iq]
    for key in settings.keys():
        beta = -np.log(res[key]['k_abs'][:, 1] / res[key]['k_abs'][:, 0])/np.log(res[key]['lam'][1] / res[key]['lam'][0])
        ax.semilogx(a, beta, '-', label=key)
        
    ax.set_ylim(0,3.5)
    ax.set_ylabel(r'opacity spectral index $\beta$')
    
    
    #### Albedo & 90º scattering probability #####
    ax = axs[2, iq]
    
    for key in settings.keys():
        line1, = ax.semilogx(a, res[key]['omega'][:, ilam], '--', alpha=0.5)
        line2, = ax.semilogx(a, res[key]['Pscat'][:, ilam], ':', c=line1.get_color(), alpha=0.5)
        line3, = ax.semilogx(a, res[key]['Pscat'][:, ilam] * res[key]['omega'][:, ilam], '-', c=line1.get_color())
    
    ax.plot([], [], c='k', ls=line1.get_linestyle(), alpha=line1.get_alpha(), label='$\omega$')
    ax.plot([], [], c='k', ls=line2.get_linestyle(), alpha=line2.get_alpha(), label='$P_{90}$')
    ax.plot([], [], c='k', ls=line3.get_linestyle(), alpha=line3.get_alpha(), label='$\omega\cdot P_{90}$')
    
    for i,key in enumerate(settings.keys()):
        line3, = ax.semilogx(a, res[key]['Pscat'][:, ilam+1] * res[key]['omega'][:, ilam+1], '-', c='k', alpha=0.25)
        
    ax.set_xlim(a[0], 0.9 * a[-1])
    ax.set_ylim(0,1.1)
    ax.legend(loc='upper right')
    ax.set_xlabel('maximum particle size, $a_\mathrm{max}$ [cm]')
    ax.set_ylabel(r'$\omega, P_{90}, \omega \cdot P_{90}$')
    
    for ax in axs.ravel():
        apply_araa_style(ax)

for ax in axs[:, 1]:
    ax.set_ylabel('')
    for label in ax.get_yticklabels():
        label.set_visible(False)
    
    f.savefig(output_dir / f'opacity.pdf', transparent=True, bbox_inches='tight')